# messing around with some neural ode's for stochastic models
- Duvenaud paper on Neural SDE's: https://arxiv.org/pdf/2001.01328.pdf
- Paper on jump processes with Neural ODE's: https://arxiv.org/abs/1905.10403

# using torchsde
- https://github.com/google-research/torchsde/blob/master/DOCUMENTATION.md

# we fit this 
dy(t) = f(t, y(t)) dt + g(t, y(t)) dW(t)        y(t0) = y0 

In [5]:
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchsde

In [6]:
batch_size, state_size, brownian_size = 32, 3, 1
t_size = 20

class SDE(torch.nn.Module):
    noise_type = 'diagonal'
    sde_type = 'ito'

    def __init__(self):
        super().__init__()
        self.mu = torch.nn.Linear(state_size, 
                                  state_size)
        self.sigma = torch.nn.Linear(state_size, 
                                     state_size * brownian_size)
        self.sigma = torch.nn.Linear(state_size, 
                                     state_size)

    def f(self, t, y):
        return self.mu(y)  # shape (batch_size, state_size)

    def g(self, t, y):
        return self.sigma(y)
        return self.sigma(y).view(batch_size, 
                                  state_size, 
                                  brownian_size)


In [7]:
sde = SDE()
y0 = torch.full((batch_size, state_size), 0.1)
ts = torch.linspace(0, 1, t_size)
# # Initial state y0, the SDE is solved over the interval [ts[0], ts[-1]].
# # ys will have shape (t_size, batch_size, state_size)

In [8]:
ys = torchsde.sdeint_adjoint(sde, y0, ts)

In [10]:
ys

tensor([[[ 0.1000,  0.1000,  0.1000],
         [ 0.1000,  0.1000,  0.1000],
         [ 0.1000,  0.1000,  0.1000],
         ...,
         [ 0.1000,  0.1000,  0.1000],
         [ 0.1000,  0.1000,  0.1000],
         [ 0.1000,  0.1000,  0.1000]],

        [[ 0.1583, -0.0331,  0.1124],
         [ 0.2452,  0.1157,  0.0543],
         [ 0.2615, -0.0324,  0.0298],
         ...,
         [ 0.1393,  0.2591,  0.0860],
         [ 0.2862,  0.2142, -0.0447],
         [ 0.0047, -0.0073,  0.0376]],

        [[ 0.2422, -0.1955,  0.0935],
         [ 0.2947,  0.2163,  0.0576],
         [ 0.2520, -0.0805, -0.2791],
         ...,
         [ 0.1467,  0.4716, -0.0671],
         [ 0.2850,  0.2204, -0.0362],
         [ 0.1532, -0.1500,  0.0103]],

        ...,

        [[-0.2465, -0.7217,  0.1909],
         [ 0.2763, -0.4019,  0.4904],
         [ 0.6046, -0.4938,  0.1655],
         ...,
         [ 0.3867,  0.4331, -0.2993],
         [-1.8486, -0.1590, -2.6492],
         [ 0.6840, -0.4568,  0.2060]],

        [[